In [ ]:
%pip install torch

In [ ]:
%pip install transformers

In [ ]:
%pip install accelerate

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')
table = db.table('articles')

articles = table.all()

print(f'loaded {len(articles)} articles')

In [ ]:
articles = [x for x in articles if x['abstract'] != 'No abstract available.']

print(f'retaining {len(articles)} articles')

In [ ]:
model_id = "mistralai/Mistral-Nemo-Instruct-2407"

## Quantized Gemma model, stored locally
# model_id = "models/google__gemma-2-27b-it"

# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

## Quantized Llama model, stored locally
# model_id = "models/meta-llama__Meta-Llama-3-70B-Instruct"

model_kwargs = {
    "low_cpu_mem_usage": True,
    "device_map": "sequential", # load the model into GPUs sequentially, to avoid memory allocation issues with balancing
    "torch_dtype": "auto"
}

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    **model_kwargs
)

In [ ]:
import utils
utils.print_model_info(model)

In [ ]:
generate_kwargs = {
    "max_new_tokens": 1024,
    "do_sample": True,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.95,
    "bos_token_id": tokenizer.bos_token_id,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": tokenizer.eos_token_id
}

prompt = """
Identify ten or more keywords from the scientific article abstract below.
Output the results in a JSON array, with each item in the array a keyword.

abstract:
"""

In [ ]:
prompt += articles[0]['abstract']

In [ ]:
%%time

messages = [
    {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    **generate_kwargs
)

response = outputs[0][input_ids.shape[-1]:]

print(tokenizer.decode(response, skip_special_tokens=True))